In [1]:
#Implementation found in https://github.com/dengjunquan/DoA-Estimation-MUSIC-ESPRIT/blob/master/DoAEstimation.py
# https://dengjunquan.github.io/posts/2018/08/DoAEstimation_Python/

import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as LA
import scipy.signal as ss
import operator
import math

############# TO DO ADD NOISE BECAUSE I REMOVED IT 

# Functions
'''
'''
def music(CovMat,L,N,array,Angles, frequency):
    # CovMat is the signal covariance matrix, L is the number of sources, N is the number of antennas
    # array holds the positions of antenna elements 
    #(array and Angles ONLY USEFUL FOR response vector so should give response vector instead)
    # Angles are the grid of directions in the azimuth angular domain
    _,V = LA.eig(CovMat)
    #print(V)
    #print("JUSTE EN HAUT")
    Qn  = V[:,L:N]
    numAngles = Angles.size
    pspectrum = np.zeros(numAngles)
    for i in range(numAngles):
        av = array[:,Angles[i],frequency]
        #print("The shape i must have in response vector")
        #print(av.shape)
        pspectrum[i] = 1/LA.norm((Qn.conj().transpose()@av))
    psindB    = np.log10(10*pspectrum/pspectrum.min())
    #print(psindB)
    DoAsMUSIC,_= ss.find_peaks(psindB, prominence=0.2)
    #print("MAX")
    #print(ss.argrelmax(psindB, order=180))
    return DoAsMUSIC,psindB


# I think not possible because it uses np.arcsin
# and arcsin is between -pi/2 and +pi/2 so several angles possible as a response
'''
def esprit(CovMat,L,N):
    # CovMat is the signal covariance matrix, L is the number of sources, N is the number of antennas
    _,U = LA.eig(CovMat)
    S = U[:,0:L]
    Phi = LA.pinv(S[0:N-1]) @ S[1:N] # the original array is divided into two subarrays [0,1,...,N-2] and [1,2,...,N-1]
    eigs,_ = LA.eig(Phi)
    DoAsESPRIT = np.arcsin(np.angle(eigs)/np.pi)
    return DoAsESPRIT
#=============================================================

def toDegrees(angle):
    degreeangle = 180*angle/math.pi
    return degreeangle
'''

lamda = 1 # wavelength
kappa = np.pi/lamda # wave number
L = 3  # number of sources
#Used everywhere need to modify it for number of elements in response vector
snr = 0 # signal to noise ratio
n_samples = 128


H_theta_time_total = np.load('lego1_h_theta_time.npy')    

# A faire mettre ça en freq et remplacer 
#le deuxieme 0 par l'angle choisi si theta ou par les angles possibles si angles

number_mics = H_theta_time_total.shape[2]
N = number_mics


H_theta_freq_total = []
for mic in range(number_mics):
    H_theta_freq_total.append(np.fft.rfft(np.transpose(H_theta_time_total[:,:,mic]), n_samples))

H_theta_freq_total = np.asarray(H_theta_freq_total)
print(H_theta_freq_total.shape)

#Getting the first frquency of the 12th angle
#H_theta_freq_test = H_theta_freq_total[:,12,1]
    
Df = H_theta_freq_total.shape[1] #number of directions for a fine discretization

array = np.linspace(0,(N-1)/2,N)
#print(array.shape)

plt.figure()
plt.subplot(221)
plt.plot(array,np.zeros(N),'^')
plt.title('Uniform Linear Array')
plt.legend(['Antenna'])

#Transform our thetas and angles in radians so it works easier 
#Thetas = np.pi*(np.random.rand(L)-1/2)   
Thetas = np.random.choice(range(Df), L, replace=False) # random source directions

#print(Thetas)

Alphas = np.random.randn(L) + np.random.randn(L)*1j # random source powers
#print(Alphas.shape)
Alphas = np.sqrt(1/2)*Alphas
#print(Alphas)

# ADAPTER avec anglesnf donc des angles en degrés 
#Angles = np.linspace(-np.pi/2,np.pi/2,360)
Angles = np.arange(Df, dtype=np.int64)*360./Df # list of angles in degrees
Angles = (Angles/2).astype(int)
#print(Angles)

numAngles = Angles.size
#print(Angles)

#print(Thetas)
#print(Alphas)
#print(array)

# A CHANGER Array_response_vector dans array surement mettre mes réponses de lego et Thetas
# theta est en radians mes response sont en degrés donc quelque chose à changer 

### Sert a rien aussi 
'''
h = np.zeros(N)
#print(array_response_vector(array,Thetas[0]))
for i in range(L):
    #print("The response is ")
    #print(array_response_vector(array,Thetas[i]))
    h = h + Alphas[i]*array_response_vector(array,Thetas[i])

# A CHANGER Array_response_vector dans array surement mettre mes réponses de lego et Thetas
# Seulement utile pour le graphe 
######
hv = np.zeros(numAngles)
for j in range(numAngles):
    av = array_response_vector(array,Angles[j])
    hv[j] = np.abs(np.inner(h,av.conj()))

powers = np.zeros(L)
for j in range(L):
    av = array_response_vector(array,Thetas[j])
    powers[j] = np.abs(np.inner(h,av.conj()))

plt.subplot(222)
plt.plot(Angles,hv)
plt.plot(Thetas,powers,'*')
plt.title('Correlation')
plt.legend(['Correlation power','Actual DoAs'])

#####
#'''

numrealization = 100
H = np.zeros((N,numrealization)) + 1j*np.zeros((N,numrealization))
#print(H.shape)
for iter in range(numrealization):
    htmp = np.zeros(N)
    #Signal generation to modify 
    for i in range(L):
        pha = np.exp(1j*2*np.pi*np.random.rand(1))
        #print(Thetas[i])        
        htmp = htmp + Alphas[i]*H_theta_freq_total[:,Thetas[i],5]
    #print(htmp.shape)
    #Adding noise
    H[:,iter] = htmp
CovMat = H@H.conj().transpose()
#print("SHAPE")
#print(CovMat.shape)

# MUSIC algorithm
DoAsMUSIC, psindB = music(CovMat,L,N,H_theta_freq_total,Angles)
#print(psindB)
#print("Ici")
'''
print(DoAsMUSIC.shape)
print(DoAsMUSIC)
print(psindB.shape)
print(psindB)
'''


plt.subplot(223)
plt.plot(Angles,psindB)
plt.plot(Angles[DoAsMUSIC],psindB[DoAsMUSIC],'x')
plt.title('MUSIC')
plt.legend(['pseudo spectrum','Estimated DoAs'])

# ESPRIT algorithm
'''
DoAsESPRIT = esprit(CovMat,L,N)
for esprit_angle_index in range(DoAsESPRIT.shape[0]):
    DoAsESPRIT[esprit_angle_index] = toDegrees(DoAsESPRIT[esprit_angle_index])
plt.subplot(224)
plt.plot(Thetas,np.zeros(L),'*')
plt.plot(DoAsESPRIT,np.zeros(L),'x')
plt.title('ESPRIT')
plt.legend(['Actual DoAs','Estimated DoAs'])
'''
print('Actual DoAs:',np.sort(Thetas),'\n')
print('MUSIC DoAs:',np.sort(Angles[DoAsMUSIC]),'\n')
#print('ESPRIT DoAs:',np.sort(DoAsESPRIT),'\n')

plt.show()

(6, 180, 65)
[138 141 147]
(3,)
[-0.87744576+0.27574511j -0.2790577 -0.63760177j -0.36796992+0.53573108j]
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179]
SHAPE
(6, 6)
[[ 0.9871401 +0.j          0.15935015-0.01272645j  0.04127759-0.00469312j
   0.03466267-0.03050322j  0.61066892+0.j         -0.04653499-0.10501744j]
 

<Figure size 640x480 with 2 Axes>